In [11]:
import pandas as pd
import warnings
import requests
warnings.filterwarnings('ignore')


In [4]:
!curl -sSL https://files.grouplens.org/datasets/movielens/ml-25m.zip > ../data/ml-25m.zip && unzip ../data/ml-25m.zip

^C


In [3]:
!(cd ../data/ && curl -O https://files.grouplens.org/datasets/movielens/ml-latest-small.zip && unzip ml-latest-small.zip)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   162k      0  0:00:05  0:00:05 --:--:--  215k
Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [1]:
spark

In [2]:
movies_spark_df = spark.read.format('csv').option('header','true').load('../data/ml-latest-small/movies.csv', sep=',')

In [4]:
movies_df = pd.read_csv('../data/ml-latest-small/movies.csv', sep=',')
tags_df = pd.read_csv('../data/ml-latest-small/tags.csv', sep=',')
links_df = pd.read_csv('../data/ml-latest-small/links.csv', sep=',')
ratings_df = pd.read_csv('../data/ml-latest-small/ratings.csv', sep=',')

In [6]:
movies_df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
fully_movie = movies_df.merge(links_df, on='movieId', suffixes=('_1', '_2'))

In [8]:
fully_movie

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,5476944,432131.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,5914996,445030.0
9739,193585,Flint (2017),Drama,6397426,479308.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,8391976,483455.0


In [9]:
fully_movie.isnull().sum()

movieId    0
title      0
genres     0
imdbId     0
tmdbId     8
dtype: int64

In [12]:
payload = {'userName': 'noing', 'password': 'noi06122001'}
session = requests.Session()
session.post('https://movielens.org/api/sessions', json=payload)

<Response [200]>

In [13]:
def get_movie_data(id):
    imdb = session.get(f'https://movielens.org/api/movies/{id}')
    movie_data = imdb.json()['data']['movieDetails']['movie']
    return movie_data

In [14]:
genre_df = pd.DataFrame(columns=['movieId', 'genre'])
laguage_df = pd.DataFrame(columns=['movieId', 'language'])
director_df = pd.DataFrame(columns=['movieId', 'director'])
actor_df = pd.DataFrame(columns=['movieId', 'actor'])


In [15]:
fully_movie.drop('genres', axis=1, inplace=True) 

In [17]:
for i in fully_movie.index:
    movie_id = fully_movie.loc[i, 'movieId']
    print(f'processing movie id {movie_id}')
    movie_data = get_movie_data(movie_id)
    fully_movie.loc[i, 'plotSummary'] = movie_data['plotSummary']
    fully_movie.loc[i, 'releaseYear'] = movie_data['releaseYear']
    fully_movie.loc[i, 'posterPath'] = movie_data['posterPath']
    fully_movie.loc[i, 'releaseDate'] = movie_data['releaseDate']
    fully_movie.loc[i, 'title'] = movie_data['title']
    fully_movie.loc[i, 'mpaa'] = movie_data['mpaa']
    for genre in movie_data['genres']:
        genre_df.append({'movieId': movie_id, 'genre': genre}, ignore_index=True)
    for language in movie_data['languages']:
        laguage_df.append({'movieId': movie_id, 'language': language}, ignore_index=True)
    for director in movie_data['directors']:
        director_df.append({'movieId': movie_id, 'director': director}, ignore_index=True)
    for actor in movie_data['actors'][:5]:
        actor_df.append({'movieId': movie_id, 'actor': actor}, ignore_index=True)
    

processing movie id 1
processing movie id 2
processing movie id 3
processing movie id 4
processing movie id 5
processing movie id 6
processing movie id 7
processing movie id 8
processing movie id 9
processing movie id 10
processing movie id 11
processing movie id 12
processing movie id 13
processing movie id 14
processing movie id 15
processing movie id 16
processing movie id 17
processing movie id 18
processing movie id 19
processing movie id 20
processing movie id 21
processing movie id 22
processing movie id 23
processing movie id 24
processing movie id 25
processing movie id 26
processing movie id 27
processing movie id 28
processing movie id 29
processing movie id 30
processing movie id 31
processing movie id 32
processing movie id 34
processing movie id 36
processing movie id 38
processing movie id 39
processing movie id 40
processing movie id 41
processing movie id 42
processing movie id 43
processing movie id 44
processing movie id 45
processing movie id 46
processing movie id 

processing movie id 389
processing movie id 390
processing movie id 391
processing movie id 393
processing movie id 405
processing movie id 406
processing movie id 407
processing movie id 408
processing movie id 409
processing movie id 410
processing movie id 412
processing movie id 413
processing movie id 414
processing movie id 415
processing movie id 416
processing movie id 417
processing movie id 418
processing movie id 419
processing movie id 420
processing movie id 421
processing movie id 422
processing movie id 423
processing movie id 424
processing movie id 425
processing movie id 426
processing movie id 427
processing movie id 428
processing movie id 429
processing movie id 430
processing movie id 431
processing movie id 432
processing movie id 433
processing movie id 434
processing movie id 435
processing movie id 436
processing movie id 437
processing movie id 438
processing movie id 440
processing movie id 441
processing movie id 442
processing movie id 444
processing movie

KeyboardInterrupt: 

In [24]:
fully_movie

,movieId,title,genres,imdbId,tmdbId,link
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,link 1
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,link 2
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,link 3
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,link 4
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,link 5
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,5476944,432131.0,link 193581
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,5914996,445030.0,link 193583
9739,193585,Flint (2017),Drama,6397426,479308.0,link 193585
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,8391976,483455.0,link 193587


In [35]:
full_df = ratings_df.join(tags_df, rsuffix='_1', on='userId', how='outer')
users_df = full_df[['userId']]
users_df.drop_duplicates(inplace=True, keep='last')
users_df.reset_index(drop=True, level=0, inplace=True)
users_df

,userId
0,1
1,2
2,3
3,4
4,5
...,...
3678,3678
3679,3679
3680,3680
3681,3681


In [41]:
import requests
import json
API_KEY = 'cfa9db117214a23524eabcd3a1517fc8'
r = requests.get(f'https://api.themoviedb.org/3/movie/550?api_key={API_KEY}')
r.json()

{'adult': False,
 'backdrop_path': '/hZkgoQYus5vegHoetLkCJzb17zJ.jpg',
 'belongs_to_collection': None,
 'budget': 63000000,
 'genres': [{'id': 18, 'name': 'Drama'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 35, 'name': 'Comedy'}],
 'homepage': 'http://www.foxmovies.com/movies/fight-club',
 'id': 550,
 'imdb_id': 'tt0137523',
 'original_language': 'en',
 'original_title': 'Fight Club',
 'overview': 'A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.',
 'popularity': 85.501,
 'poster_path': '/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg',
 'production_companies': [{'id': 508,
   'logo_path': '/7cxRWzi4LsVm4Utfpr1hfARNurT.png',
   'name': 'Regency Enterprises',
   'origin_country': 'US'},
  {'id': 711,
   'logo_path': '/tEiIH5QesdheJmDAqQwvtN60727.png',
   'na

In [72]:
category_df = pd.DataFrame(columns=['movieId', 'genre'])
category_df = category_df.append({'movieId': 1, 'genre': 'asd'}, ignore_index=True)
category_df

/tmp/ipykernel_17223/1918181218.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  category_df = category_df.append({'movieId': 1, 'genre': 'asd'}, ignore_index=True)


,movieId,genre
0,1,asd


In [73]:
payload = {'userName': 'noing', 'password': 'noi06122001'}
session = requests.Session()
session.post('https://movielens.org/api/sessions', json=payload)
imdb = session.get('https://movielens.org/api/movies/318')
movie_data = imdb.json()['data']['movieDetails']['movie']
movie_data

{'movieId': 318,
 'tmdbMovieId': 278,
 'imdbMovieId': '0111161',
 'title': 'The Shawshank Redemption',
 'originalTitle': 'The Shawshank Redemption',
 'mpaa': 'R',
 'runtime': 142,
 'releaseDate': '1994-09-23',
 'dvdReleaseDate': '1999-12-21',
 'genres': ['Drama', 'Crime'],
 'languages': ['English'],
 'directors': ['Frank Darabont'],
 'actors': ['Tim Robbins',
  'Morgan Freeman',
  'Bob Gunton',
  'William Sadler',
  'Clancy Brown',
  'Gil Bellows',
  'Mark Rolston',
  'James Whitmore',
  'Jeffrey DeMunn',
  'Larry Brandenburg',
  'Neil Giuntoli',
  'Brian Libby',
  'David Proval',
  'Joseph Ragno',
  'Jude Ciccolella',
  'Paul McCrane',
  'Renee Blaine',
  'Scott Mann',
  'John Horton',
  'Gordon Greene',
  'Alfonso Freeman',
  'V.J. Foster',
  'John E. Summers',
  'Frank Medrano',
  'Mack Miles',
  'Alan R. Kessler',
  'Morgan Lund',
  'Cornell Wallace',
  'Gary Lee Davis',
  'Neil Summers',
  'Ned Bellamy',
  'Joe Pecoraro',
  'Harold E. Cope Jr.',
  'Brian Delate',
  'Don McManus',


In [75]:
movie_data['plotSummary']

'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.'

https://image.tmdb.org/t/p/w500/hBcY0fE9pfXzvVaY4GKarweriG2.jpg